In [18]:
# data cleaning 

import re 
import argparse
from pathlib import Path
from pathlib import Path 
import sentencepiece as spm 



class CorpusBuilder:
    # cleans and prepares text dat for toekniier trainng 
    # what it does 
    #  remvoes junk char 
    # keeps only good line 


    def __init__(self, min_chars=5):
        self.min_chars = min_chars
        self.control_chars = re.compile(r"[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]")
        self.multiple_spaces = re.compile(r"[ \t\u00A0]+")

    def clean_line(self, text):
        # zero width space 
        text = text.replace('\u200b',"")
        # remove bom byte oreder mark k
        text = text.replace("\uffeff","")
        # remove contril characters 
        text = self.control_chars.sub("",text)
        # remove extraa spaces at start/en 
        text = text.strip()
        return text 
    

    # def build_corpus(self, input_files, output_file):
    #     # c

In [19]:
# tokenizer trainer 

class TokenizerTrainer:
    # trains a SP tokenizer on your corpus 
    def __init__(self, vocab_size = 16000):
        self.vocab_size = vocab_size

    def train(self, corpus_file, output_dir, model_name="tokenizer"):

        # create op direc
        output_path = Path(output_dir)
        output_path.mkdir(parents=True, exist_ok=True)

        # full path for model 
        model_prefix = output_path / model_name

        print("trining.........")
        print("-"*69)
        print(f"corpus :{corpus_file}")
        print(f"Vocab_size: {self.vocab_size}")
        print(f"model op:{model_prefix}.model")


        spm.SentencePieceTrainer.Train(
            input = str(corpus_file),
            model_prefix = str(model_prefix),

            # setting 
            model_type = "bpe", # tying bpe at first might change latter
            vocab_size = self.vocab_size,
            character_coverage  = 0.9995, #chtgpt said to put this 

            # special toksn (fixed)
            unk_id=0,  # Unknown token
            bos_id=1,  # Beginning of sequence
            eos_id=2,  # End of sequence  
            pad_id=3,  # Padding token

            # performance (specially for large data)
            shuffle_input_sentence=True,
            input_sentence_size=2000000,
            train_extremely_large_corpus=False,  

        )

        print("-"*69)
        print("Done traininng")
        print("Model Saved")
        print("vocab saved")

            






In [17]:
# tokenizer*( for using the trained model)

class NepaliTokenizer:

    def __init__(self, model_path):
        # load a train tokenizer 

        # checking if file exists 
        if not Path(model_path).exists():
            raise FileExistsError(f"FIle xina babu:{model_path}")
        
        # load spm model 
        self.sp = spm.SentencePieceProcessor()
        sucess = self.sp.Load(str(model_path))

        if not sucess:
            raise RuntimeError(f"Failed to laod model")
        
        # get special token IDS 
        self.unk_id = self.sp.unk_id()
        self.bos_id = self.sp.bos_id()
        self.eos_id = self.sp.eos_id()
        self.pad_id = self.sp.pad_id()

        print("TOkeinzer loaded")
        print("Vocab size: {self.vocab_size}")

    def vocab_size(self):
        return self.sp.GetPieceSize()
    
    def encode(self, text, add_bos= False, add_eos = False):
        # convert text to token ids 
        # text  Input : नमस्ते
        # add bos : add begnign token at stat 
        # add_eos : add end token at end
        # returns  token idss

        # get token ids from sp 
        ids = self.sp.EncodeAsIds(text)
        
        # Build result
        result = []
        
        # Add beginning token if requested
        if add_bos:
            result.append(self.bos_id)
        
        # Add all token IDs
        for token_id in ids:
            result.append(token_id)
        
        # Add end token if requested
        if add_eos:
            result.append(self.eos_id)
        
        return result
    

    def decode(self, ids, remove_special = True):
        # vice versa of encodder 
        # filter specal tokens if requurested 
        if remove_special:
            filtered = []
            for token_id in ids:
                # skipping special tokens 
                if token_id == self.bos_id:
                    continue
                if token_id == self.eos_id:
                    continue
                if token_id == self.pad_id:
                    continue

                # keeping regular tokens
                filtered.append(token_id)
            ids = filtered

        # decode to text 
        return self.sp.DecodeIds(ids)
    
    def encode_batch(self, texts, add_bos=False, add_eos=False):
        
        # Encode multiple texts at once.
        
        results = []
        for text in texts:
            encoded = self.encode(text, add_bos=add_bos, add_eos=add_eos)
            results.append(encoded)
        return results
    
    def decode_batch(self, ids_list, remove_special=True):
       
        # Decode multiple token sequences at once.
        results = []
        for ids in ids_list:
            decoded = self.decode(ids, remove_special=remove_special)
            results.append(decoded)
              
    




In [23]:
trainer = TokenizerTrainer(vocab_size=100)
trainer.train(corpus_file=corpus_path, output_dir="tokenizer_test", model_name="nepali_bpe")


trining.........
---------------------------------------------------------------------
corpus :sample_nepali_corpus.txt
Vocab_size: 100
model op:tokenizer_test/nepali_bpe.model
---------------------------------------------------------------------
Done traininng
Model Saved
vocab saved


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: sample_nepali_corpus.txt
  input_format: 
  model_prefix: tokenizer_test/nepali_bpe
  model_type: BPE
  vocab_size: 100
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 2000000
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: 3
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_di

In [24]:
tokenizer = NepaliTokenizer("tokenizer_test/nepali_bpe.model")

text = "जीवन कविता हो ।"

ids = tokenizer.encode(text, add_bos=True, add_eos=True)
print("Encoded IDs:", ids)

decoded = tokenizer.decode(ids)
print("Decoded text:", decoded)


TOkeinzer loaded
Vocab size: {self.vocab_size}
Encoded IDs: [1, 21, 22, 19, 8, 2]
Decoded text: जीवन कविता हो ।


In [25]:
texts = [
    "म पहाडमा बस्छु ।",
    "कविता मेरो आत्मा हो ।"
]

encoded_batch = tokenizer.encode_batch(texts, add_bos=True, add_eos=True)
print("Batch encoded:", encoded_batch)

decoded_batch = tokenizer.decode_batch(encoded_batch)
print("Batch decoded:", decoded_batch)


Batch encoded: [[1, 16, 38, 31, 5, 73, 0, 85, 95, 99, 8, 2], [1, 22, 47, 49, 19, 8, 2]]
Batch decoded: None
